In [ ]:
import pandas as pd

# Read and shape data
data = pd.read_excel('data.xlsx', sheet_name='PS19A_PERSON__SmplR10K', keep_default_na=False)
data.shape

# Add headers to data frame
addresses = data['person_address']
fullFrame = pd.DataFrame(data, columns=addresses, index=addresses)

In [ ]:
import Levenshtein as lev
import multiprocessing
import threading
import math

# Number of threads to create
threadCount = multiprocessing.cpu_count();

# Use full data frame
data = fullFrame
# Use partial data frame of n addresses
#n = 128
#data = fullFrame.iloc[:n, :n]

# Amount of addresses to use per thread
threadAddressCount = int(math.ceil(float(len(data)) / threadCount));

class CalculateDistanceThread(threading.Thread):
    def __init__(self, threadID):
        threading.Thread.__init__(self)
        self.threadID = threadID
    def run(self):
        start = threadAddressCount * self.threadID
        for n in range(threadAddressCount):
            if (start + n < len(data)):
                for x in data.index:
                    ratio = lev.ratio(data.index[start + n].lower(), x.lower())
                    fullFrame.at[data.index[start + n], x] = ratio

# Create a list with n threads
threads = []
for n in range(threadCount):
    thread = CalculateDistanceThread(n)
    threads.append(thread)

# Start all threads
for thread in threads:
    thread.start();

# Wait for all threads to finish
for thread in threads:
    thread.join();

In [ ]:
fullFrame.head(8)